In [1]:
import pandas as pd

In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import pytz
import requests
from datetime import datetime, timedelta
import json
import streamlit as st
import base64
import time
import requests
import xgboost as xgb
import joblib


class PySimFin:
    def __init__(self, api_key):
        self.__api_key = api_key
        self.headers = {
            "accept": "application/json",
            "Authorization": f"{self.__api_key}"  
        }

    def get_stock_prices(self, tickers: list, days: int):
        base_url = "https://backend.simfin.com/api/v3/companies/prices/compact" 
        start_date = (datetime.today() - timedelta(days=days)).strftime('%Y-%m-%d')
        ticker_list = ",".join(tickers)
        url = f"{base_url}?ticker={ticker_list}&start={start_date}"
        response = requests.get(url, headers=self.headers)
        data = json.loads(response.text)
        all_data = []

        for company_data in data:
            columns = company_data['columns']
            rows = company_data['data']
            ticker = company_data['ticker']
            df = pd.DataFrame(rows, columns=columns)
            df['Ticker'] = ticker
            all_data.append(df)

        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    
    def get_stock_prices_backtest(self, tickers: list, start_date, end_date):
        base_url = "https://backend.simfin.com/api/v3/companies/prices/compact" 
        start_date = start_date.strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        ticker_list = ",".join(tickers)
        url = f"{base_url}?ticker={ticker_list}&start={start_date}"
        response = requests.get(url, headers=self.headers)
        data = json.loads(response.text)
        all_data = []

        for company_data in data:
            columns = company_data['columns']
            rows = company_data['data']
            ticker = company_data['ticker']
            df = pd.DataFrame(rows, columns=columns)
            df['Ticker'] = ticker
            all_data.append(df)

        final_df = pd.concat(all_data, ignore_index=True)
        final_df['Date'] = pd.to_datetime(final_df['Date'])  # Ensure Date column is in datetime format
        final_df = final_df.sort_values(by=["Ticker", "Date"])

        # Filter by the date range
        final_df = final_df.loc[(final_df['Date'] >= start_date) & (final_df['Date'] <= end_date)]

        return final_df
    
    def get_predictions_data_backtest(self, start_date, end_date):
        tickers = ["AAPL", "MSFT", "BRO", "FAST", "ODFL"]
        batch_size = 2
        all_data = []

        for i in range(0, len(tickers), batch_size):
            batch = tickers[i:i + batch_size]
            df_tickers = self.get_stock_prices_backtest(batch, start_date, end_date)
            df_tickers = df_tickers.rename(columns={
                "Date": "Date", "Opening Price": "Open", "Highest Price": "High",
                "Lowest Price": "Low", "Last Closing Price": "Close", 
                "Trading Volume": "Volume", "Adjusted Closing Price": "Adj. Close",
                "Common Shares Outstanding": "Shares Outstanding"
            })
            all_data.append(df_tickers)
            time.sleep(0.3)

        # Combine all data
        df_tickers = pd.concat(all_data)

        pivoted_df = df_tickers.pivot(index="Date", columns="Ticker", values=["Open", "High", "Low", "Close", "Adj. Close", "Volume"])

        # Flatten column names
        pivoted_df.columns = [f"{col[0]}_{col[1]}" for col in pivoted_df.columns]
        pivoted_df.reset_index(inplace=True)

        # Merge with original dataset to ensure each ticker retains its own target variable
        merged_df = df_tickers.merge(pivoted_df, on="Date", how="left")

        cols_to_drop = ["Open", "High", "Low", "Close", "Adj. Close", "Volume", "Dividend Paid"]
        merged_df.drop(columns=cols_to_drop, inplace=True)
        return merged_df
    




In [3]:
from backtesting import Backtest, Strategy
import pandas as pd
import joblib
import xgboost as xgb
from datetime import datetime

class SignalStrategy(Strategy):
    """
    A strategy that follows the given signals with TP and SL.
    Buys at signal 1 and sells at signal 0, waiting for TP/SL to be hit.
    """
    # Define default tp and sl as class variables, but we'll override them later
    tp = 0.005  # Default take profit percentage
    sl = 0.005  # Default stop loss percentage

    def init(self):
        # Initialize the strategy with take profit and stop loss parameters
        self.tp_percent = self.tp  # Access tp directly
        self.sl_percent = self.sl  # Access sl directly
        self.signal = self.data.signal
        self.entry_price = None

    def next(self):
        # Called on every new bar
        if self.position:
            # We have an open position
            # Calculate TP and SL prices
            tp_price = self.entry_price * (1 + self.tp_percent)
            sl_price = self.entry_price * (1 - self.sl_percent)

            # Check for TP or SL exit
            if self.data.High[-1] >= tp_price:
                self.position.close()
                self.entry_price = None
            elif self.data.Low[-1] <= sl_price:
                self.position.close()
                self.entry_price = None
            elif self.signal[-1] == 0:  # Sell signal
                self.position.close()
                self.entry_price = None
        else:
            # No open position
            if self.signal[-1] == 1:  # Buy signal
                self.buy()
                self.entry_price = self.data.Close[-1]

def backtest(ticker, start_date, end_date, cash, tp, sl):
    API_KEY = "0ce27565-392d-4c49-a438-71e3b39f298f"
    simfin = PySimFin(API_KEY)
    df = simfin.get_stock_prices_backtest(ticker, start_date, end_date)
   # merged_df = simfin.get_predictions_data_backtest()
    merged_df = simfin.get_predictions_data_backtest(start_date, end_date)


    # Load the pre-trained XGBoost model
    model = joblib.load("xgb.joblib")
    X = merged_df.drop(columns=['Date', 'Ticker'])

    # Generate predictions
    dmat = xgb.DMatrix(X)
    predictions = model.predict(dmat)
    predictions = (predictions > 0.5).astype(int)

    df_predictions = df.copy()
    merged_df['Prediction'] = predictions
    merged_df = merged_df[merged_df['Ticker'].isin(ticker)]
    df_predictions['signal'] = merged_df['Prediction'].values
    df_predictions = df_predictions.rename(columns={
        'Opening Price': 'Open',
        'Highest Price': 'High',
        'Lowest Price': 'Low',
        'Last Closing Price': 'Close',
        'Trading Volume': 'Volume',
        'Adjusted Closing Price': 'Adj. Close',
        'Common Shares Outstanding': 'Shares Outstanding'
    })

    df_predictions = df_predictions.drop(columns=['Dividend Paid'])
    df_predictions = df_predictions[['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adj. Close', 'Volume', 'Shares Outstanding', 'signal']]

    bck = df_predictions[df_predictions['Ticker'].isin(ticker)]
    bck['Date'] = pd.to_datetime(bck['Date'])
    bck.set_index('Date', inplace=True)

    # Run backtest with the strategy and provided TP/SL values
    bt = Backtest(bck, SignalStrategy, cash=cash, commission=.002)
    
    # Pass the custom TP/SL values to the strategy using the 'run' method
    stats = bt.run(tp=tp, sl=sl)
    bt.plot()
    print(stats)

    return df_predictions

# Run the backtest
start_date = datetime(2019, 3, 29)
end_date = datetime(2025, 3, 11)
df = backtest(['MSFT'], start_date, end_date, 10000, tp=0.1, sl=0.05)


Start                     2019-03-29 00:00:00
End                       2025-03-11 00:00:00
Duration                   2174 days 00:00:00
Exposure Time [%]                     99.3984
Equity Final [$]                  19522.57346
Equity Peak [$]                   26845.64456
Commissions [$]                     4821.0869
Return [%]                           95.22573
Buy & Hold Return [%]               222.57928
Return (Ann.) [%]                    11.92851
Volatility (Ann.) [%]                30.80814
CAGR [%]                              8.06317
Sharpe Ratio                          0.38719
Sortino Ratio                         0.62589
Calmar Ratio                          0.23868
Alpha [%]                          -100.85048
Beta                                  0.88093
Max. Drawdown [%]                   -49.97693
Avg. Drawdown [%]                    -3.79464
Max. Drawdown Duration      941 days 00:00:00
Avg. Drawdown Duration       35 days 00:00:00
# Trades                          

In [47]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd

class SignalStrategy(Strategy):
    """
    A strategy that follows the given signals with TP and SL.
    Buys at signal 1 and sells at signal 0, waiting for TP/SL to be hit.
    """
    tp_percent = 0.05  # 5% take profit
    sl_percent = 0.03  # 3% stop loss

    def init(self):
        # Initialize the strategy
        self.signal = self.data.signal
        self.entry_price = None

    def next(self):
        # Called on every new bar
        if self.position:
            # We have an open position
            # Calculate TP and SL prices
            tp_price = self.entry_price * (1 + self.tp_percent)
            sl_price = self.entry_price * (1 - self.sl_percent)

            # Check for TP or SL exit
            if self.data.High[-1] >= tp_price:
                self.position.close()
                self.entry_price = None
            elif self.data.Low[-1] <= sl_price:
                self.position.close()
                self.entry_price = None
            elif self.signal[-1] == 0:  # Sell signal
                self.position.close()
                self.entry_price = None
        else:
            # No open position
            if self.signal[-1] == 1:  # Buy signal
                self.buy()
                self.entry_price = self.data.Close[-1]



In [31]:
from backtesting import Backtest, Strategy
import pandas as pd

class SignalStrategy(Strategy):
    """
    A strategy that follows the given signals with TP and SL.
    Buys at signal 1 and sells at signal 0, waiting for TP/SL to be hit.
    """
    tp_percent = 0.03  # 5% take profit
    sl_percent = 0.02  # 3% stop loss

    def init(self):
        self.signal = self.data.signal

    def next(self):
        if self.position:
            # Exit if sell signal appears
            if self.signal[-1] == 0:
                self.position.close()
        else:
            # Buy if signal is 1
            if self.signal[-1] == 1:
                sl = self.data.Close[-1] * (1 - self.sl_percent)
                tp = self.data.Close[-1] * (1 + self.tp_percent)
                self.buy(sl=sl, tp=tp)



/var/folders/6v/qvykl7vd2yb8gqx3qb10rc5m0000gn/T/ipykernel_93722/3256039829.py:33: UserWarning: 'Can't superimpose OHLC data with rule 'None'(index datetime resolution: 'nanosecond'). Skipping.
  bt.plot()


Start                     1970-01-01 00:00:00
End                       1970-01-01 00:00...
Duration                  0 days 00:00:00....
Exposure Time [%]                    99.73262
Equity Final [$]                   9123.12285
Equity Peak [$]                   14229.49757
Commissions [$]                   14484.48389
Return [%]                           -8.76877
Buy & Hold Return [%]               299.72891
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
CAGR [%]                                  NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Alpha [%]                          -185.68488
Beta                                  0.59025
Max. Drawdown [%]                   -59.80451
Avg. Drawdown [%]                    -3.08181
Max. Drawdown Duration    0 days 00:00:00....
Avg. Drawdown Duration    0 days 00:00:00....
# Trades                          

In [25]:
1064+431

1495